In [ ]:
import os
import sys



# Parsloes Park forest

Area: 10000 sq meter
Density: 3.2 m2
Total Saplings: 32,000

**Canopy Layer:**
1 Oak, english Quercus robur 3200 (quantity)
2 Oak, sessile Qusrcus petrasa -3200 
3 Cherry., wild Prunus avium -1300 
4 Beech, common Fagus sylvatica -600 
5 Lime, larged leaved Tilis cordsta  - 1000 

**Tree Layer:**
6 Birch, downy Botula pubescens -2200
7 Wild service tree Sorbus torminalis -600 
8 Birch, silver Betula penduls 2600 
9 Cherry. bird Prunus padus -1300 

**Sub Tree Layer**
10 Hazel Corvlus avellana -1600
11 Dogwood Cormus sanguinea -1300 
12 Maple. field Acer campestre -1300 
13 Willow, goat Sslix capree -600
14 Hornbeam Carpinus betulus 1000 
15 Apple, crab Malus syvestns 600
16 Spindle Euonymus Buropasa- 600
17 Holly liex -1000
18 Whiteboarn Sorbus aria 600 
19 Elder Sambucus nigra 1000 
20 Box, common Buxus sempervirens - 600 
21 Juniper Junipenus communis 2200

**Shrub Layer:**
22 Shrubs Blackthorn Prunus - 1600 
23 Shrubs Wayfaring tree Viburmum lantana 1000 
24 Shrubs Guelder, rose Viburmum gpulus -1000 

TOTAL 32,000 saplings:

In [2]:
import pandas as pd
import folium

# Create a sample dataframe
data = {
    'lat': [51.5074, 48.8566, 40.7128],
    'lng': [-0.1278, 2.3522, -74.0060],
    'name': ['London', 'Paris', 'New York']
}
df = pd.DataFrame(data)

# Create a folium map object
m = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=3)

# Plot points
for index, row in df.iterrows():
    folium.Marker([row['lat'], row['lng']], popup=row['name']).add_to(m)

# Save or show map
m.save("sample.html")

Your map should have been opened in your browser automatically.
Press ctrl+c to return.
